In [1]:
import numpy as np

In [2]:
class logreg:
    def __init__(self, gd_type: str = 'full', tolerance: float = 1e-4, 
                 max_iter: int = 10000, eta: float = 1e-2) -> None:
        """
          gd_type: type of gradient descent ('full'/'stochastic')
          tolerance: threshold for stopping gradient descent
          max_iter: maximum number of steps in gradient descent
          eta: learning rate
        """
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.eta = eta
        self.w = None
        self.loss_history = None 
        self.cur_iter = 0 #current number of iterations
        
    def fit(self, X: np.array, y: np.array) -> None:
        #fit the model on training data and save loss value after each iteration.
        y = y.reshape(-1, 1)
        self.loss_history = []
        self.w = np.array([1.0]*X.shape[1]) #initialization of weights
        self.w = self.w.reshape(-1, 1)
        while self.cur_iter <= self.max_iter:
            gradient = self.calc_gradient(X, y)
            a1 = np.sum(self.w*self.w) #euclidean norm before step
            self.w -= self.eta * gradient
            a2 = np.sum(self.w*self.w) #euclidean norm after step
            loss = self.calc_loss(X, y)
            self.loss_history.append([loss, self.cur_iter])
            self.cur_iter += 1
            
            if abs(a1-a2)<=self.tolerance:
              break
    
    def predict_proba(self, X: np.array) -> np.array:
        #calculate probability of positive and negative class for each observation
        if self.w is None:
            raise Exception('not trained yet')
        a = 1/(1+np.exp(X @ self.w)) #first column
        b = 1/(1+np.exp(-X @ self.w)) #2nd
        a = pd.DataFrame(a)
        b = pd.DataFrame(b)
        a = a.rename(columns={0:'false'})
        b = b.rename(columns={0:'true'})
        res = pd.concat([a,b], axis=1)
        return res
    
    def predict(self, X: np.array) -> np.array:
        #predict class for each observation
        if self.w is None:
            raise Exception('not trained yet')
        a = 1/(1+np.exp(X @ self.w))
        a = np.around(a)
        return pd.DataFrame(a)
    
    def calc_gradient(self, X: np.array, y: np.array) -> np.array:
        #alculate gradient of loss function after each iteration
        y = y.reshape(-1, 1)
        if self.gd_type == 'stochastic':
            rrr = random.choice(np.arange(X.shape[0]))
            grad = X[rrr].T @ (1/(1+np.exp(-X[rrr] @ self.w)) - y[rrr])/X.shape[0]
        elif self.gd_type == 'full':
            grad = X.T @ (1/(1+np.exp(-X @ self.w)) - y)/X.shape[0]
        return grad
    
    def calc_loss(self, X: np.array, y: np.array) -> float:
        #calculate value of loss function after each iteration
        y = y.reshape(-1, 1)
        return -(1/X.shape[0])*(np.sum(y*(np.log(1/(1+np.exp(X @ self.w)))) + (1-y)*np.log(1/(1+np.exp(X @ self.w)))))
    
    def graph_loss(self):
        #draw total loss
        df = pd.DataFrame(self.loss_history)
        plt.figure(figsize=(16, 7))
        plt.plot(df.loc[:][1], df.loc[:][0])
        plt.title('log loss (number of steps)')
        plt.show()
    